# Open Soure Threat Intel Lookups using Requests API

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Open-Soure-Threat-Intel-Lookups-using-Requests-API" data-toc-modified-id="Open-Soure-Threat-Intel-Lookups-using-Requests-API-1">Open Soure Threat Intel Lookups using Requests API</a></span><ul class="toc-item"><li><span><a href="#Description-:" data-toc-modified-id="Description-:-1.1">Description :</a></span></li><li><span><a href="#Import-Python-Packages" data-toc-modified-id="Import-Python-Packages-1.2">Import Python Packages</a></span></li></ul></li><li><span><a href="#ThreatIntel-Providers-Research" data-toc-modified-id="ThreatIntel-Providers-Research-2">ThreatIntel Providers Research</a></span><ul class="toc-item"><li><span><a href="#TI-Feed-Summary-with-rate-limit-details" data-toc-modified-id="TI-Feed-Summary-with-rate-limit-details-2.1">TI Feed Summary with rate limit details</a></span></li><li><span><a href="#Supported-Providers-in-this-Notebook" data-toc-modified-id="Supported-Providers-in-this-Notebook-2.2">Supported Providers in this Notebook</a></span></li></ul></li><li><span><a href="#IP-Enrichments" data-toc-modified-id="IP-Enrichments-3">IP Enrichments</a></span></li><li><span><a href="#Threat-Intel-Lookups-from-supported-Providers" data-toc-modified-id="Threat-Intel-Lookups-from-supported-Providers-4">Threat Intel Lookups from supported Providers</a></span><ul class="toc-item"><li><span><a href="#ThreatCrowd" data-toc-modified-id="ThreatCrowd-4.1">ThreatCrowd</a></span><ul class="toc-item"><li><span><a href="#Inline-HTML-graph-view-from-ThreatCrowd-Site" data-toc-modified-id="Inline-HTML-graph-view-from-ThreatCrowd-Site-4.1.1">Inline HTML graph view from ThreatCrowd Site</a></span></li></ul></li><li><span><a href="#XForce" data-toc-modified-id="XForce-4.2">XForce</a></span></li><li><span><a href="#PulseDive" data-toc-modified-id="PulseDive-4.3">PulseDive</a></span></li><li><span><a href="#ThreatMiner" data-toc-modified-id="ThreatMiner-4.4">ThreatMiner</a></span></li><li><span><a href="#Dshield" data-toc-modified-id="Dshield-4.5">Dshield</a></span></li><li><span><a href="#Alienvault" data-toc-modified-id="Alienvault-4.6">Alienvault</a></span></li></ul></li></ul></div>

## Description :
Notebook is example implementaion of open source threat intel lookup using just requests API. You can also install individual python modules given in below table but using requests is more convenient. You can make it modular by declaring a ipaddress variable on the top and calling with each individual variable.  

Notebook is not meant to be extensive reference and aimed for a very basic implementation. 
Have a look at Microsoft msticpy library and example TIProvider Notebook for a more structured imlementation with more features. 

 - Library : https://pypi.org/project/msticpy/
 - TIProvider Notebook: https://nbviewer.jupyter.org/github/microsoft/msticpy/blob/master/docs/notebooks/TIProviders.ipynb

## Import Python Packages

In [1]:
# Imports
from IPython import get_ipython
from IPython.display import display, HTML

import pandas as pd

import requests, json
from requests.exceptions import HTTPError
from requests.auth import HTTPBasicAuth

# ThreatIntel Providers Research

## TI Feed Summary with rate limit details
![title](images/OpenSourceTIFeedSummary.png)

## Supported Providers in this Notebook

 - Shodan : 
     - API Registeration : https://account.shodan.io/register
 - Alienvault OTX
    - API Registeration : https://otx.alienvault.com/api
 - IBM Xforce
    - API Registration : https://www.ibm.com/account/reg/signup?formid=urx-30243 
 - ThreatMiner
    - API Registration : https://threatminer.org/api.php
 - ThreatCrowd
    - API Registration : https://github.com/AlienVault-OTX/ApiV2
 - RiskIQ
    - API Registration : https://community.riskiq.com/registration
 - Pulsedive
    - API Registration : https://pulsedive.com/auth/new.php
 - Censys
    - API Registration : https://censys.io/register
 - SANS DShield
    - API Registration : https://isc.sas.edu/api

In [2]:
# Fill in API Keys by registering the respective TI provider
# Consider other secure ways to pass these values via environment variables or reading from files
IPSTACK_API=''
CYMON_API=''
SHODAN_API=''
OTX_API=''
XFORCE_API_KEY =''
XFORCE_API_PASSWORD=''
RISKIQ_API = ''
PULSEDIVE_API=''
CENSYS_API_ID=''
CENSYS_API_SECRET=''
SAFEBROWSING_API=''

# IP Enrichments

In [3]:
#Determine IP Address type based on IP ranges.
def ip_type(string):
    """ function to populate IPType category(e.g. Private/Public/Multicast)"""
    if ip.ip_address(string).is_private:
        return 'Private'
    elif ip.ip_address(string).is_multicast:
        return 'Multicast'
    elif ip.ip_address(string).is_unspecified:
        return 'Unspecified'
    elif ip.ip_address(string).is_reserved:
        return 'Reserved'
    elif ip.ip_address(string).is_loopback:
        return 'Loopback'
    elif ip.ip_address(string).is_global:
        return 'Public'
    elif ip.ip_address(string).is_link_local:
        return 'Link Local'
    else:
        return 'Unknown'

# Threat Intel Lookups from supported Providers

In [4]:
def shodan_iplookup(string):
    """ function to IP lookup using Shodan API """
    API_BASE_URL="https://api.shodan.io/shodan/host/"
#     df = pd.DataFrame(columns=['TISource', 'Status', 'Category','References'])
    try:
        result =  requests.get(API_BASE_URL + string + "?key=" + SHODAN_API)
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err

def ipstack_whoislookup(ipaddress):
    """ Perform whois Lookups on IP Address using IPStack API """
    filtered_fields = 'country_code,country_name,latitude,longitude'
    api = 'http://api.ipstack.com/' + ipaddress + '?access_key=' + ipstack_access_key +'&fields='+filtered_fields 
    result = urllib.request.urlopen(api).read()
    result = result.decode()
    result = json.loads(result)

    return result

def threatcrowd_iplookup(string):
    """ Perform whois Lookups on IP Address using Threatcrowd API """
    API_BASE_URL="https://www.threatcrowd.org/searchApi/v2/ip/report/"
    try:
        result =  requests.get(API_BASE_URL, params = {"ip": string})
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err
    
def otx_iplookup(string,path="v1/indicators/IPv4/",section="/general"):
    """ Perform whois Lookups on IP Address using Alienvault OTX API """
    API_BASE_URL="https://otx.alienvault.com/api/"
    try:
        result =  requests.get(API_BASE_URL+path+string+section)
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err
    
def dshield_iplookup(string,outputformat="?json"):
    """ Perform whois Lookups on IP Address using SANS DShield API """
    API_BASE_URL="https://isc.sans.edu/api/ip/"
    try:
        result =  requests.get(API_BASE_URL+string+outputformat)
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err

def threatminer_iplookup(string,rt=1):
    """ Perform whois Lookups on IP Address using Threatminer API """
    API_BASE_URL="https://api.threatminer.org/v2/host.php"
    try:
        result =  requests.get(API_BASE_URL,params = {"q": string,"rt":rt})
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err

def pulsedive_iplookup(string):
    """ Perform whois Lookups on IP Address using Pulsedive API """
    API_BASE_URL="https://pulsedive.com/api/info.php"
    try:
        result =  requests.get(API_BASE_URL,params = {"indicator": string,"key":PULSEDIVE_API})
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err

def censys_iplookup(string,path="/view/ipv4/"):
    """ Perform whois Lookups on IP Address using Censys API """
    API_BASE_URL="https://censys.io/api/v1"
    try:
        result =  requests.get(API_BASE_URL+path+string,auth=(CENSYS_API_ID, CENSYS_API_SECRET))
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err

def riskiq_iplookup(string,path="/v2/enrichment"):
    """ Perform whois Lookups on IP Address using RiskIQ Community API """
    API_BASE_URL = 'https://api.passivetotal.org'
    URL= API_BASE_URL+path+"?query="+string
    try:
        result =  requests.get(URL, auth=(RISKIQ_USERID, RISKIQ_API))
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err
    
def xforce_iplookup(string,path="ipr/"):
    """ Perform whois Lookups on IP Address using IBM Xforce API """
    API_BASE_URL="https://api.xforce.ibmcloud.com/"
    URL= API_BASE_URL+path+string
    try:
        result =  requests.get(URL, auth=HTTPBasicAuth(XFORCE_API_KEY, XFORCE_API_PASSWORD))
        return result.json()
    except HTTPError as http_err:
        return http_err
    except Exception as err:
        return err

## ThreatCrowd

In [5]:
threatcrowdresults=threatcrowd_iplookup('128.210.157.251')

#Code To extract essential details from raw json response
if threatcrowdresults['response_code'] == '1' : 
    display(HTML('<div class="alert alert-danger">Match Found via Alienvault API </div>'))
    print(f"\nThreatCrowd References:\n")
    for item in threatcrowdresults['references']: print(item)
    print(f"\n HTML Graph can be viewed on link:\n {threatcrowdresults['permalink']}")
else:
    display(HTML('<div class="alert alert-success">No Match Found via ThreatCrowd API </div>'))


ThreatCrowd References:

http://www.talosintelligence.com/feeds/ip-filter.blf
https://otx.alienvault.com/pulse/56fdd2f84637f207cbcd02da/%7Chttp://rules.emergingthreats.net/blockrules/emerging-botcc.rules
https://otx.alienvault.com/pulse/56fdd4c24637f207cacce53b/%7Chttps://zeustracker.abuse.ch/blocklist.php?downloadbadips

 HTML Graph can be viewed on link:
 https://www.threatcrowd.org/ip.php?ip=128.210.157.251


### Inline HTML graph view from ThreatCrowd Site

In [6]:
from IPython.display import IFrame
IFrame(src='https://www.threatcrowd.org/ip.php?ip=128.210.157.251', width=1024, height=760)

## XForce

In [7]:
xforceresults=xforce_iplookup("192.64.119.63")

#Code To extract essential details from raw json response
if len(xforceresults['cats']) > 0 : 
    display(HTML('<div class="alert alert-danger"> Match Found via Xforce API </div>'))
    print(f"\n Xforce Tags or Categories Identified:\n\t{xforceresults['cats']}")
    print(f"\n Xforce Category Description:\n{xforceresults['categoryDescriptions']}")
else:
    display(HTML('<div class="alert alert-success">No Match Found via Xforce API </div>'))


 Xforce Tags or Categories Identified:
	{'Anonymisation Services': 43}

 Xforce Category Description:
{'Anonymisation Services': 'This category contains IP addresses of Web proxies (websites that allow the user to anonymously view websites). Furthermore, IP addresses are listed that can be used directly to surf anonymously (e.g. by adding them to the browser configuration).'}


## PulseDive

In [8]:
pulsediveresults=pulsedive_iplookup("192.64.119.63")

#Code To extract essential details from raw json response
if len(pulsediveresults['threats']) > 0 : 
    display(HTML('<div class="alert alert-danger"> Match Found via Pulsedive API </div>'))
    print(f"{len(pulsediveresults['threats'])} Threats Idenfied")
    print(f"\n PulseDive Tags or Categories Identified:")
    for item in pulsediveresults['threats']:print(item['category'])
    print(f"\n PulseDive Category Description:")
    for item in pulsediveresults['threats']:print(item['name'])
    print(f"\n PulseDive Feed Names:")
    for item in pulsediveresults['feeds']:print(item['name'])
else:
    display(HTML('<div class="alert alert-success">No Match Found via Pulsedive API </div>'))

3 Threats Idenfied

 PulseDive Tags or Categories Identified:
malware
phishing
malware

 PulseDive Category Description:
Banjori
Phishing
Suppobox

 PulseDive Feed Names:
C&C IPs
ZeroDot1's Bad IPs Feed


## ThreatMiner

In [9]:
threatminerresults=threatminer_iplookup("35.35.35.35",rt=6)

#Code To extract essential details from raw json response
if threatminerresults['status_message'] == 'Results found.': 
    display(HTML('<div class="alert alert-danger">Match Found via ThreatMiner API </div>'))
    print(f"{len(threatminerresults['results'])} References Idenfied")
    print(f"\n ThreatMiner File along with URL References:\n")
    for item in threatminerresults['results']:print("FileName Reference:\t"+item['filename']+ "\nURL Reference :\t"+item['URL'])
else:
     display(HTML('<div class="alert alert-success">No Match Found via ThreatMiner API </div>'))

6 References Idenfied

 ThreatMiner File along with URL References:

FileName Reference:	Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf
URL Reference :	https://www.threatminer.org/report.php?q=Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf&y=2018
FileName Reference:	Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf
URL Reference :	https://www.threatminer.org/report.php?q=Area-1-Security-OperationDoos-IRN2TargetsSaudiArabianOilandGasIndustryWithCareer-ThemedPhishingAttack.pdf&y=2018
FileName Reference:	DNSTunnelingintheWild_OverviewofOilRig’sDNSTunneling.pdf
URL Reference :	https://www.threatminer.org/report.php?q=DNSTunnelingintheWild_OverviewofOilRig’sDNSTunneling.pdf&y=2019
FileName Reference:	TheOilRigCampaign_AttacksonSaudiArabianOrganizationsDeliverHelminthBackdoor-PaloAltoNetworksBlogPaloAltoNetworksBlog.pdf


## Dshield

In [10]:
dshieldresults=dshield_iplookup("128.210.157.251")

#Code To extract essential details from raw json response
if 'threatfeeds' in dshieldresults['ip'].keys(): 
    display(HTML('<div class="alert alert-danger">Match Found via DShield API </div>'))
    print(f"{len(dshieldresults['ip']['threatfeeds'])} ThreatFeeds Idenfied")
    print(f"\n Dshield ThreatFeeds Results:\n {dshieldresults['ip']['threatfeeds']}")
else:
    display(HTML('<div class="alert alert-success">No Match Found via DShiled API </div>'))

1 ThreatFeeds Idenfied

 Dshield ThreatFeeds Results:
 {'zeuscc': {'lastseen': '2019-07-08', 'firstseen': '2013-09-29'}}


## Alienvault

In [11]:
otxresults=otx_iplookup('109.248.222.47')

#Code To extract essential details from raw json response
if 'pulse_info' in otxresults.keys(): 
    display(HTML('<div class="alert alert-danger">Match Found via Alienvault API</div>'))
#     print("Match Found via Alienvault API")
    print(f"{len(otxresults['pulse_info']['pulses'])} Pulses Idenfied")
    print(f"\nAlienvault Tags identified:")
    for item in otxresults['pulse_info']['pulses']: print(f"Tags idenfied {item['tags']} from {item['name']}") 
    print(f"\nReferences identified:")
    for item in otxresults['pulse_info']['pulses']: print(item['references'])
else:
    display(HTML('<div class="alert alert-success">No Match Found via Alienvault API </div>'))

2 Pulses Idenfied

Alienvault Tags identified:
Tags idenfied [] from Locky Ransomware C2 IP blocklist (LY_C2_IPBL)
Tags idenfied ['ransomware'] from Ransomware IP Blocklist 3/21/2019

References identified:
[]
['https://ransomwaretracker.abuse.ch/downloads/RW_IPBL.txt']


In [12]:
otxresults=otx_iplookup('192.168.168.81')

#Code To extract essential details from raw json response
if 'pulse_info' in otxresults.keys(): 
    display(HTML('<div class="alert alert-danger">Match Found via Alienvault API</div>'))
#     print("Match Found via Alienvault API")
    print(f"{len(otxresults['pulse_info']['pulses'])} Pulses Idenfied")
    print(f"\nAlienvault Tags identified:")
    for item in otxresults['pulse_info']['pulses']: print(f"Tags idenfied {item['tags']} from {item['name']}") 
    print(f"\nReferences identified:")
    for item in otxresults['pulse_info']['pulses']: print(item['references'])
else:
    display(HTML('<div class="alert alert-success">No Match Found via Alienvault API </div>'))